# Analisando os dados do SIM

In [1]:
from pysus.ftp.databases.sim import SIM

ModuleNotFoundError: No module named 'pysus'

In [2]:
sim = SIM().load() # Loads the files from DATASUS

In [3]:
sim.metadata

{'long_name': 'Sistema de Informação sobre Mortalidade',
 'source': 'http://sim.saude.gov.br',
 'description': ''}

In [28]:
sim.groups

{'CID10': 'DO', 'CID9': 'DOR'}

In [29]:
sim.paths

(/dissemin/publicos/SIM/CID10/DORES, /dissemin/publicos/SIM/CID9/DORES)

## Baixando dados específicos

In [4]:
group = 'CID10'
uf = 'SP'

year = [i for i in range(2014, 2021+1)]
# month = [i for i in range(1, 12+1)]

# year = 2022

In [6]:
files = sim.get_files(group, uf = uf, year = year)

files

[DOSP2014.dbc,
 DOSP2015.dbc,
 DOSP2016.dbc,
 DOSP2017.dbc,
 DOSP2018.dbc,
 DOSP2019.dbc,
 DOSP2020.dbc,
 DOSP2021.dbc]

In [7]:
sim.describe(files)

AttributeError: 'list' object has no attribute 'path'

In [2]:
stagePath = '/home/ferdanielinux/desktop-linux/scripts-python/proj-aires/Dados/SIM/stage/parquet'

In [9]:
sim.download(files, local_dir = stagePath)

DOSP2021.parquet: 100%|██████████| 1.06M/1.06M [01:45<00:00, 10.0kB/s]


[/home/ferdanielinux/desktop-linux/scripts-python/proj-aires/Dados/SIM/stage/parquet/DOSP2014.parquet,
 /home/ferdanielinux/desktop-linux/scripts-python/proj-aires/Dados/SIM/stage/parquet/DOSP2015.parquet,
 /home/ferdanielinux/desktop-linux/scripts-python/proj-aires/Dados/SIM/stage/parquet/DOSP2016.parquet,
 /home/ferdanielinux/desktop-linux/scripts-python/proj-aires/Dados/SIM/stage/parquet/DOSP2017.parquet,
 /home/ferdanielinux/desktop-linux/scripts-python/proj-aires/Dados/SIM/stage/parquet/DOSP2018.parquet,
 /home/ferdanielinux/desktop-linux/scripts-python/proj-aires/Dados/SIM/stage/parquet/DOSP2019.parquet,
 /home/ferdanielinux/desktop-linux/scripts-python/proj-aires/Dados/SIM/stage/parquet/DOSP2020.parquet,
 /home/ferdanielinux/desktop-linux/scripts-python/proj-aires/Dados/SIM/stage/parquet/DOSP2021.parquet]

## Lendo dados SIM

In [2]:
import pandas as pd

import os

In [3]:
# Definir o caminho do diretório
directory = r'C:\Users\Fernando\Desktop\Compromissos\IC - Banco de Dados\Dados\CID-10'

directory = directory.replace("\\", "/")

print(directory)


C:/Users/Fernando/Desktop/Compromissos/IC - Banco de Dados/Dados/CID-10


In [4]:
# Entrando na pasta e selecionando o arquivo
directoryFull = os.path.join(directory, 'CID-10-SUBCATEGORIAS.csv')
    
directoryFull

'C:/Users/Fernando/Desktop/Compromissos/IC - Banco de Dados/Dados/CID-10\\CID-10-SUBCATEGORIAS.csv'

In [8]:
cidParquet = pd.read_csv(directoryFull, sep = ';', encoding = 'latin1')

cidParquet.head()

,SUBCAT,CLASSIF,RESTRSEXO,CAUSAOBITO,DESCRICAO,DESCRABREV,REFER,EXCLUIDOS
0,A000,NaN,NaN,NaN,"Cólera devida a Vibrio cholerae 01, biótipo ch...",A00.0 Colera dev Vibrio cholerae 01 biot cholerae,NaN,NaN
1,A001,NaN,NaN,NaN,"Cólera devida a Vibrio cholerae 01, biótipo El...",A00.1 Colera dev Vibrio cholerae 01 biot El Tor,NaN,NaN
2,A009,NaN,NaN,NaN,Cólera não especificada,A00.9 Colera NE,NaN,NaN
3,A010,NaN,NaN,NaN,Febre tifóide,A01.0 Febre tifoide,NaN,NaN
4,A011,NaN,NaN,NaN,Febre paratifóide A,A01.1 Febre paratifoide A,NaN,NaN


In [10]:
print(f"O arquivo possui {cidParquet.shape[0]} linhas e {cidParquet.shape[1]} colunas.")

O arquivo possui 12451 linhas e 8 colunas.


In [11]:
#Verificando quais colunas possuem valores nulos
cidParquet.isnull().sum()


SUBCAT            0
CLASSIF       12073
RESTRSEXO     11577
CAUSAOBITO    11160
DESCRICAO         0
DESCRABREV        0
REFER         12294
EXCLUIDOS     12435
dtype: int64

In [13]:
# Removendo as colunas que possuem valores nulos
newDf = cidParquet.dropna(axis = 1)

newDf.head()

,SUBCAT,DESCRICAO,DESCRABREV
0,A000,"Cólera devida a Vibrio cholerae 01, biótipo ch...",A00.0 Colera dev Vibrio cholerae 01 biot cholerae
1,A001,"Cólera devida a Vibrio cholerae 01, biótipo El...",A00.1 Colera dev Vibrio cholerae 01 biot El Tor
2,A009,Cólera não especificada,A00.9 Colera NE
3,A010,Febre tifóide,A01.0 Febre tifoide
4,A011,Febre paratifóide A,A01.1 Febre paratifoide A


## Inserindo os dados no banco de dados

### Conectando ao banco

In [14]:
from sqlalchemy import create_engine

In [15]:
# Configurações da conexão
usuario = 'postgres'
senha = 'dan313852*'
host = 'localhost'
porta = '5432'
banco_de_dados = 'sim_database'

# URL de conexão
url = f"postgresql://{usuario}:{senha}@{host}:{porta}/{banco_de_dados}"

engine = create_engine(url)

# Teste a conexão
try:
    connection = engine.connect()
    print("Conexão bem-sucedida!")
except Exception as e:
    print(f"Erro ao conectar: {e}")
finally:
    connection.close()


Conexão bem-sucedida!


In [18]:
import sqlalchemy as sa

# PAREI AQUI

In [22]:
# Fazendo um cast para os dados nas colunas. SUBCAT é um varchar com tamanho 4, DESCRICAO é um varchar com tamanho 100 e DESCRABREV é um varchar com tamanho 50
newDf['SUBCAT'] = newDf['SUBCAT'].astype('str')
newDf['DESCRICAO'] = newDf['DESCRICAO'].astype('str')
newDf['DESCABREV'] = newDf['DESCRABREV'].astype('str')

# Salvando o dataframe no banco de dados com o cast do tipo de dados
newDf.to_sql('cid10', con = engine, if_exists = 'replace', index = False, dtype = {'SUBCAT': sa.types.VARCHAR(length = 4), 'DESCRICAO': sa.types.VARCHAR(length = 100), 'DESCRABREV': sa.types.VARCHAR(length = 50})

SyntaxError: closing parenthesis '}' does not match opening parenthesis '(' (2471913005.py, line 7)

### Concatenando algumas amostras de cada ano

In [ ]:
# Removi a variável "CRM", já que ela não está presente nos dados a partir de 2019
colunas_desejadas = [
    "CONTADOR", "ORIGEM", "TIPOBITO", "DTOBITO", "HORAOBITO", "NATURAL",
    "CODMUNNATU", "DTNASC", "IDADE", "SEXO", "RACACOR", "ESTCIV", "ESC", 
    "ESC2010", "SERIESCFAL", "OCUP", "CODMUNRES", "LOCOCOR", "CODESTAB", 
    "ESTABDESCR", "CODMUNOCOR", "IDADEMAE", "ESCMAE", "ESCMAE2010", 
    "SERIESCMAE", "OCUPMAE", "QTDFILVIVO", "QTDFILMORT", "GRAVIDEZ", 
    "SEMAGESTAC", "GESTACAO", "PARTO", "OBITOPARTO", "PESO", "TPMORTEOCO", 
    "OBITOGRAV", "OBITOPUERP", "ASSISTMED", "EXAME", "CIRURGIA", "NECROPSIA", 
    "LINHAA", "LINHAB", "LINHAC", "LINHAD", "LINHAII", "CAUSABAS", "CB_PRE", 
    "COMUNSVOIM", "DTATESTADO", "CIRCOBITO", "ACIDTRAB", "FONTE", "NUMEROLOTE", 
    "TPPOS", "DTINVESTIG", "CAUSABAS_O", "DTCADASTRO", "ATESTANTE", "STCODIFICA", 
    "CODIFICADO", "VERSAOSIST", "VERSAOSCB", "FONTEINV", "DTRECEBIM", 
    "ATESTADO", "DTRECORIGA", "CAUSAMAT", "ESCMAEAGR1", "ESCFALAGR1", 
    "STDOEPIDEM", "STDONOVA", "DIFDATA", "NUDIASOBCO", "NUDIASOBIN", 
    "DTCADINV", "TPOBITOCOR", "DTCONINV", "FONTES", "TPRESGINFO", "TPNIVELINV", 
    "NUDIASINF", "DTCADINF", "MORTEPARTO", "DTCONCASO", "FONTESINF", "ALTCAUSA"
]

In [25]:
# Quantidade de amostras desejadas
num_samples = 1000

# Criar uma nova lista com amostragem de linhas e seleção de colunas
sampled_dataframes = [
    df[colunas_desejadas].sample(n=min(num_samples, len(df)), random_state=42)
    for df in dataFrameList
]

# Concatenar os DataFrames amostrados em um único DataFrame
final_dataframe = pd.concat(sampled_dataframes, ignore_index=True)

In [27]:
final_dataframe.shape[1]

87

In [28]:
final_dataframe.to_sql('concatenated_data', engine, if_exists='replace', index=True)

96

## Uma breve tratativa nos dados

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
# Configurações da conexão
usuario = 'postgres'
senha = 'dan313852*'
host = 'localhost'
porta = '5432'
banco_de_dados = 'sim_database'

# URL de conexão
url = f"postgresql://{usuario}:{senha}@{host}:{porta}/{banco_de_dados}"

engine = create_engine(url)

# Teste a conexão
try:
    connection = engine.connect()
    print("Conexão bem-sucedida!")
except Exception as e:
    print(f"Erro ao conectar: {e}")
finally:
    connection.close()


Conexão bem-sucedida!


In [3]:
df = pd.read_sql_table('concatenated_data', engine)

In [7]:
print(f"O DataFrame possui {df.shape[0]} linhas e {df.shape[1]} colunas.")

df.head()

O DataFrame possui 9000 linhas e 88 colunas.


,index,CONTADOR,ORIGEM,TIPOBITO,DTOBITO,HORAOBITO,NATURAL,CODMUNNATU,DTNASC,IDADE,...,DTCONINV,FONTES,TPRESGINFO,TPNIVELINV,NUDIASINF,DTCADINF,MORTEPARTO,DTCONCASO,FONTESINF,ALTCAUSA
0,0,205490,1,2,22092014,0940,829,292740,25081949,465,...,,,,,,,,,XXXXXXX,
1,1,39787,1,2,19022014,0600,835,351380,15061993,420,...,,,,,,,,,XXXXXXX,
2,2,45039,1,2,27022014,1745,835,355580,24031978,435,...,,,,,,,,,XXXXXXX,
3,3,112993,1,2,29052014,1550,835,352180,14091921,492,...,,,,,,,,,XXXXXXX,
4,4,232356,1,2,26102014,1435,835,353500,22081930,484,...,,,,,,,,,XXXXXXX,


### Removendo espaços em branco

In [8]:
## Removendo espaços em branco à esquerda e à direita
df_filtered = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

print(f"O DataFrame possui {df_filtered.shape[0]} linhas e {df_filtered.shape[1]} colunas.")

O DataFrame possui 9000 linhas e 88 colunas.


In [14]:
## Listando a coluna e o número máxima de caracteres em cada uma delas
max_length = df_filtered.apply(lambda x: x.astype(str).str.len().max())

In [16]:
for coluna, tamanho in max_length.items():
    print(f"{coluna}: {tamanho}")

index: 4
CONTADOR: 7
ORIGEM: 1
TIPOBITO: 1
DTOBITO: 8
HORAOBITO: 4
NATURAL: 3
CODMUNNATU: 6
DTNASC: 8
IDADE: 3
SEXO: 1
RACACOR: 1
ESTCIV: 1
ESC: 1
ESC2010: 1
SERIESCFAL: 1
OCUP: 6
CODMUNRES: 6
LOCOCOR: 1
CODESTAB: 7
ESTABDESCR: 40
CODMUNOCOR: 6
IDADEMAE: 2
ESCMAE: 1
ESCMAE2010: 1
SERIESCMAE: 1
OCUPMAE: 6
QTDFILVIVO: 2
QTDFILMORT: 2
GRAVIDEZ: 1
SEMAGESTAC: 2
GESTACAO: 1
PARTO: 1
OBITOPARTO: 1
PESO: 4
TPMORTEOCO: 1
OBITOGRAV: 1
OBITOPUERP: 1
ASSISTMED: 1
EXAME: 1
CIRURGIA: 1
NECROPSIA: 1
LINHAA: 20
LINHAB: 20
LINHAC: 15
LINHAD: 15
LINHAII: 30
CAUSABAS: 4
CB_PRE: 0
COMUNSVOIM: 6
DTATESTADO: 8
CIRCOBITO: 1
ACIDTRAB: 1
FONTE: 1
NUMEROLOTE: 8
TPPOS: 1
DTINVESTIG: 8
CAUSABAS_O: 4
DTCADASTRO: 8
ATESTANTE: 1
STCODIFICA: 1
CODIFICADO: 1
VERSAOSIST: 6
VERSAOSCB: 3
FONTEINV: 1
DTRECEBIM: 8
ATESTADO: 50
DTRECORIGA: 8
CAUSAMAT: 0
ESCMAEAGR1: 2
ESCFALAGR1: 2
STDOEPIDEM: 1
STDONOVA: 1
DIFDATA: 4
NUDIASOBCO: 4
NUDIASOBIN: 4
DTCADINV: 8
TPOBITOCOR: 1
DTCONINV: 8
FONTES: 6
TPRESGINFO: 1
TPNIVELINV: 1
NUD

In [21]:
## Inserindo os dados no banco de dados
df_filtered.to_sql('clean_data', engine, if_exists='replace', index=False)

96

In [22]:
max_length2 = df_filtered.apply(lambda x: x.astype(str).str.len().max())
max_length2

index         4
CONTADOR      7
ORIGEM        1
TIPOBITO      1
DTOBITO       8
             ..
DTCADINF      8
MORTEPARTO    1
DTCONCASO     8
FONTESINF     7
ALTCAUSA      1
Length: 88, dtype: int64

In [23]:
# Inserindo max_length2 no banco de dados
max_length2.to_sql('max_length', engine, if_exists='replace', index=True)

88